In [121]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score


import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense

from model_new import train_and_export_models

from get_json import reduce_columns_train, process_folder, check_and_create_columns, iterate_files

from get_diff import calculate_event_differences

from clean_preprocess import preprocess_pred


import pickle

# IMPORT DATA

In [123]:
folder_path_train = "../data/Match_Diff"

In [124]:
data_dict = iterate_files(folder_path_train)
data_dict



{'DIAMOND':              matchId  target  jungleMinionsKilled  minionsKilled  totalGold  \
 0     LA2_1295998351       1                   -4             26       2016   
 1     LA2_1297406314       1                   -3              8       4442   
 2     LA2_1273457046       1                   16             -4       -590   
 3     LA2_1296091346       0                  -12            -15       -715   
 4     LA2_1281845995       1                   -8             39       1088   
 ...              ...     ...                  ...            ...        ...   
 4900  LA2_1285568894       0                   -4            -21      -1950   
 4901  LA2_1305200498       0                    8             -5       1036   
 4902  LA2_1293816263       0                   12            -15       1130   
 4903  LA2_1267892380       0                    1            -51      -5009   
 4904  LA2_1285662717       1                   -8             -6        712   
 
         xp  magicDamageDon

# STARTS MODEL

In [125]:

data_dict_read={}

for key, df in data_dict.items():
    
    data_dict_read[key] = reduce_columns_train(data_dict[key])
    


In [126]:
from clean_preprocess import preprocess_train

data_preprop = {}

for key, df in data_dict_read.items():
    
    X = data_dict_read[key].drop(columns="target")
    
    y = data_dict_read[key]["target"]

    X_preprop,transformer = preprocess_train(X)
    
    data_preprop[key] = [X_preprop,y,transformer]

In [127]:
data_preprop

{'DIAMOND': [      killType_KILL_ACE  killType_KILL_FIRST_BLOOD  killType_KILL_MULTI  \
  0                   0.0                       -1.0                  1.0   
  1                   0.0                       -1.0                  1.0   
  2                   0.0                        0.0                  0.0   
  3                   0.0                       -1.0                  0.0   
  4                   0.0                        0.0                  0.0   
  ...                 ...                        ...                  ...   
  4900                0.0                        0.0                 -1.0   
  4901                0.0                        0.0                 -1.0   
  4902                0.0                        0.0                  0.0   
  4903                0.0                       -1.0                 -1.0   
  4904                0.0                        0.0                  0.0   
  
        minionsKilled  monsterType_AIR_DRAGON  monsterType_CHE

In [128]:
league = "DIAMOND"
columns_of_interest = data_preprop[league][0].columns.tolist()
columns_of_interest

['killType_KILL_ACE',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'minionsKilled',
 'monsterType_AIR_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'totalGold',
 'towerType_INNER_TURRET',
 'towerType_OUTER_TURRET']

# LOG MODEL

In [129]:
diccionario =train_and_export_models(data_preprop)
diccionario

DIAMOND
Best Hyperparameters:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.7043819056284935
Model exported as DIAMOND_model.pkl


{'DIAMOND': LogisticRegression(C=1, max_iter=5000, solver='liblinear')}

# PREDICTION

In [130]:
pred_folder = "../data/predict"


In [134]:

minute = 10
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"]
folder_path = pred_folder
league="DIAMOND"
pickle_file_path = f"{league}_model.pkl"
with open(pickle_file_path, "rb") as file:
    # Load the data from the pickle file
    fitted_model = pickle.load(file)

def prediction(pred_folder,minute,look_events,columns_of_interest,league, fitted_model):
    df = process_folder(pred_folder,minute,look_events)
    df_dif = calculate_event_differences(df)
    df_dif.drop(columns="matchId",inplace=True)
    df_cc = check_and_create_columns(df_dif, columns_of_interest)
    X_pred_prep = preprocess_pred(df_cc,data_preprop[str(league)][2])
    model = fitted_model
    #result = model.predict(X_pred_prep)
    proba = model.predict_proba(X_pred_prep)
    
    return  proba

In [136]:
fitted_model

LogisticRegression(C=1, max_iter=5000, solver='liblinear')

In [137]:
prediction(pred_folder, minute, look_events, columns_of_interest, league, fitted_model)

/home/andres/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


array([[0.24640716, 0.75359284]])